<a href="https://www.kaggle.com/code/kohkaichunsamuel/deepseek-tir?scriptVersionId=293696484" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Setting Up VLLM Package to Run the LLM

We require to set up the v-LLM package that runs on Python 3.12.12.

Source from: https://www.kaggle.com/code/haradibots/installing-vllm-offline-fix

In [1]:
# import os
# !mkdir /kaggle/working/vllm_312_wheels
# # This downloads exactly what Python 3.12 needs
# !pip download vllm==0.9.2 -d /kaggle/working/vllm_312_wheels
# # Zip it so you can download it easily
# !zip -r vllm_312.zip /kaggle/working/vllm_312_wheels

In [2]:
import os
print(os.listdir('/kaggle/input'))

['vllm-0-9-2-offline-installer', 'aimo-vllm-accelerated-tot-sc-deepseekmath', 'deepseek-math', 'ai-mathematical-olympiad-progress-prize-3', 'vllm-whl', 'ai-mathematical-olympiad-prize', 'math-shepherd-mistral-7b-prm']


In [3]:
!find /kaggle/working -name "vllm_312_wheels"

/kaggle/working/vllm_312_wheels


In [4]:
# Install directly from the folder shown in your screenshot
!pip install --no-index --find-links=/kaggle/working/vllm_312_wheels vllm

Looking in links: /kaggle/working/vllm_312_wheels


In [5]:
# !pip uninstall -y torch
# # !pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm

### Testing the VLLM

In [6]:
import vllm
print(f"vLLM version {vllm.__version__} successfully installed!")

vLLM version 0.9.2 successfully installed!


In [7]:
!python --version

Python 3.12.12


In [8]:
import transformers
print(transformers.__version__)

4.53.3


In [9]:
# !pip install "transformers<4.54.0"

In [10]:
from vllm import LLM, SamplingParams
import pandas as pd
from tqdm import tqdm
import gc
import re
import sys
import subprocess
from collections import defaultdict, Counter
import numpy as np
from transformers import (AutoModelForCausalLM,
    AutoTokenizer,
    set_seed)
import torch
import math

llm = LLM(model="/kaggle/input/deepseek-math",
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.99,
          swap_space=4,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)

llm

2026-01-24 16:03:49.345718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769270629.367392     610 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769270629.374187     610 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769270629.391672     610 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769270629.391691     610 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769270629.391693     610 computation_placer.cc:177] computation placer alr

INFO 01-24 16:03:54 [__init__.py:244] Automatically detected platform cuda.
INFO 01-24 16:04:10 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
WARNING 01-24 16:04:10 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 01-24 16:04:10 [config.py:1472] Using max model len 2048
WARNING 01-24 16:04:10 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 01-24 16:04:10 [config.py:1593] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. Meanwhile, it may cause accuracy drop without a proper scaling factor
WARNING 01-24 16:04:13 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 01-24 16:04:13 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='/kaggle/input/deepseek-math', specu

[W124 16:04:14.052079388 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W124 16:04:14.052784548 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 01-24 16:04:27 [default_loader.py:272] Loading weights took 12.84 seconds
INFO 01-24 16:04:28 [model_runner.py:1203] Model loading took 12.8726 GiB and 13.040472 seconds
INFO 01-24 16:04:30 [worker.py:294] Memory profiling takes 1.73 seconds
INFO 01-24 16:04:30 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.99) = 14.59GiB
INFO 01-24 16:04:30 [worker.py:294] model weights take 12.87GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.95GiB; the rest of the memory reserved for KV Cache is 0.74GiB.
INFO 01-24 16:04:31 [executor_base.py:113] # cuda blocks: 202, # CPU blocks: 1092
INFO 01-24 16:04:31 [executor_base.py:118] Maximum concurrency for 2048 tokens per request: 1.58x
INFO 01-24 16:04:36 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 7.70 seconds


### Instantiating the Tokeniser for the Process Reward Model (PRM)

The PRM was obtained from: https://www.kaggle.com/datasets/bsmit1659/math-shepherd-mistral-7b-prm

In [11]:
tokenizer = llm.get_tokenizer()

good_token = '+'
bad_token = '-'
step_tag = 'ки'

prm_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm')

prm_tokenizer

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

LlamaTokenizerFast(name_or_path='/kaggle/input/math-shepherd-mistral-7b-prm', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

### Encoding the PRM

In [12]:
prm_candidate_tokens = prm_tokenizer.encode(f"{good_token} {bad_token}")[1:] # [648, 387]
step_tag_id = prm_tokenizer.encode(f"{step_tag}")[-1] # 12902

step_tag_id

12902

### Initialising the PRM

In [13]:
prm_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm',
                                                 torch_dtype=torch.float16,
                                                 device_map="balanced_low_0").eval()
prm_model

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [14]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv')
df.head()

,id,problem
0,000aaa,What is $1-1$?
1,111bbb,What is $0\times10$?
2,222ccc,Solve $4+x=4$ for $x$.


In [15]:
if len(df) < 5:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    PRIVATE = False
df.head()

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211


### Building the Naive Parser

This function only takes out the final answer (number) from the long answer the model generates

In [16]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

### Top N Strings

In [17]:
import re
import sys
import os
import subprocess

def top_n_strings(input_list, n):
    # Sort the list based on the prob values in descending order
    sorted_list = sorted(input_list, key=lambda x: x[1], reverse=True)

    # Get the top n elements
    top_n = sorted_list[:n]

    # Extract the strings from the top n elements
    result = [item[0] for item in top_n]

    return result

## Helper Functions for Phase 2 - Tool Integrated Reasoning (TIR)

### Process Output

Cuts the tiny code out and saves into a code.py block to run. If it fails, it gives an error.

In [18]:
def process_output(output):
    result = output
    
    try:
        code = output.split('```')[1][7:]

        with open('code.py', 'w') as fout:
            fout.write(code)

        batcmd = 'timeout 7 ' + sys.executable + ' code.py'
        try:
            shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
            print(shell_output)
            code_output = round(float(eval(shell_output))) % 1000
        except:
            code_output = -1
            
        if os.path.exists('code.py'):
            os.remove('code.py')

        print('CODE RESULTS', code_output)
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        code_output = -1
    
    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        result_output = -1
    
    return result_output, code_output

### Function for Process Reward Model (PRM)

- Takes all the different ideas the AI has, and gives each one a score (good or bad).
  

In [19]:
batch_size = 1

def eval_prm(candidates):
    # Initialize a list to store all the log probabilities
    all_log_probs = []

    # Process the candidates in batches
    for i in range(0, len(candidates), batch_size):
        # Select a batch of candidates
        batch_candidates = candidates[i:i + batch_size]

        # Encode all candidates into a batch of input IDs
        encoded_inputs = [prm_tokenizer.encode(candidate, return_tensors="pt") for candidate in batch_candidates]

        # Pad the encoded inputs to the same length
        max_length = max([input_id.shape[1] for input_id in encoded_inputs])  # Find the longest sequence
        padded_inputs = [
            torch.nn.functional.pad(input_id, (0, max_length - input_id.size(1)), value=prm_tokenizer.pad_token_id) for
            input_id in encoded_inputs]
        input_ids = torch.cat(padded_inputs, dim=0).to("cuda:1")  # Concatenate the padded inputs into a tensor

        with torch.no_grad():
            logits = prm_model(input_ids).logits[:, :, prm_candidate_tokens]

            scores = logits.softmax(dim=-1)[:, :, 0].squeeze()

            # Extract log probabilities for the specific candidate tokens
            log_probs = scores.log()

            if batch_size == 1:
                batch_log_probs = log_probs[-1].flatten()
            else:
                batch_log_probs = log_probs[:, -1].flatten()

            # Collect the log probabilities from this batch
            all_log_probs.extend(batch_log_probs.cpu().tolist())

    return all_log_probs

### Getting Stop Words

In [20]:
stop_words = [tokenizer.eos_token if tokenizer is not None and tokenizer.eos_token is not None else '</s>']
stop_words.append("\n")

tool_stop_words = [tokenizer.eos_token if tokenizer is not None and tokenizer.eos_token is not None else '</s>']
tool_stop_words.append("```output")

print("Stop words:", stop_words)
print("Tool stop words:", tool_stop_words)

Stop words: ['<｜end▁of▁sentence｜>', '\n']
Tool stop words: ['<｜end▁of▁sentence｜>', '```output']


In [21]:
sampling_params = SamplingParams(temperature=1.0,
                                 max_tokens=256,
                                 stop=stop_words)

tool_sampling_params = SamplingParams(temperature=1.0,
                                      max_tokens=2048,
                                      stop=tool_stop_words)


### Prompts for DeepSeek to Solve

In [22]:
cot_instruction = "\nPlease reason step by step, and put your final \
answer within \\boxed{}."

tool_instruction = '\nPlease integrate natural language reasoning \
with programs to solve the problem above, and put your final answer \
within \\boxed{}.'

### The Main Loop

1. It lets the AI solve the problem multiple times, attaining about 48 different solutions.
2. It uses the Judge to keep the best ones and eliminate the bad ones.
3. If the AI is stuck, it will switch to Python coding using the Python interpreter. (TIR is used here).
4. Finally, it counts all the answers it got and picks the one that appeared the most (majority voting).

In [ ]:
n = 1
all_prompts = []
total_results = []
total_answers = []

for i in tqdm(range(len(df))):
    id_ = df['id'].loc[i]
    problem = df['problem'].loc[i]

    messages = [
        {
            "role": "user",
            "content": problem + cot_instruction
        }
    ]

    base_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )
    current_level = 0

    current_level_nodes = [(base_prompt, 0)]  # Tuple of (node, cumulative_logprob)
    completed_paths = []

    while len(completed_paths) < n:
        # Prepare batch for generation
        batch_prompts = [node for node, _ in current_level_nodes]
        batch_responses = llm.generate(batch_prompts*16, sampling_params)  # Generate for all nodes in a batch

        prm_inputs = []  # To collect all candidates for reward model evaluation
        mapping = []  # To map back reward model scores to corresponding nodes

        # Collect candidates for reward model evaluation
        for parent_node, parent_cum_logprob in current_level_nodes:
            for candidate in batch_responses:
                if parent_node + candidate.outputs[0].text + "\n" not in [prm_input[1] for prm_input in prm_inputs]:
                    new_node = parent_node + candidate.outputs[0].text + "\n"
                    cumulative_tokens = len(candidate.prompt_token_ids) + len(candidate.outputs[0].token_ids)
                    prm_inputs.append((new_node[:-1] + " " + step_tag, new_node, parent_cum_logprob, cumulative_tokens))
                    mapping.append(len(prm_inputs) - 1)  # Store the index for mapping back the score

        # Batch reward model evaluation
        prm_scores = eval_prm([prm_input for prm_input, _, _, _ in prm_inputs])
        next_level_nodes = []

        # Distribute candidates back to their parent nodes
        for idx, (_, node, parent_cum_logprob, cumulative_tokens) in enumerate(prm_inputs):
            score = prm_scores[idx]  # Get the corresponding score
            new_cum_logprob = parent_cum_logprob + score  # Update cumulative log probability

            # Check for completions and sufficient score
            if "```" in node or "\\boxed" in node.split("\n\n")[-1]:
                completed_paths.append((node, new_cum_logprob))
            elif score > math.log(0.8) and cumulative_tokens <= 2000:  # Threshold check
                next_level_nodes.append((node, new_cum_logprob))

        # Prune to keep only the top 'n' candidates based on cumulative log probability
        next_level_nodes.sort(key=lambda x: x[1], reverse=True)  # Sort nodes by their cumulative log probability
        current_level_nodes = next_level_nodes[:n]  # Keep only the top 'n' nodes
        current_level += 1

        # If we already have 'n' completed paths, no need to continue
        if len(completed_paths) >= n:
            break
        if not current_level_nodes or current_level > 20:
            if not completed_paths:
                messages = [
                    {
                        "role": "user",
                        "content": problem + tool_instruction
                    }
                ]

                base_prompt = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False
                ) + "```python\n"

                raw_outputs = llm.generate([base_prompt]*8, tool_sampling_params)

                for response in raw_outputs:
                    completed_paths.append("```python\n" + response.outputs[0].text)
            break

    results = []
    answers = []

    for path in completed_paths:
        try:
            if "```python\n" in path:
                result_output, code_output = process_output(path)
                gc.collect()
                
                results.append(code_output)
                answers.append(code_output)
            else:
                raw_output = path[0].split("\n\n")[-1]
                result_output, code_output = process_output(raw_output)
                gc.collect()

                results.append(result_output)
                answers.append(result_output)

        except Exception as e:
            print(e)
            result_output, code_output = -1, -1
            logprob = -10000

            results.append(result_output)
            answers.append(result_output)

    total_results.append(results)
    total_answers.append(answers)

  0%|          | 0/10 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

### Main Loop

### Obtain Final Answers

In [ ]:
print(f"DataFrame rows: {len(df)}")
print(f"Total answer sets: {len(total_answers)}")
print(f"Total result sets: {len(total_results)}")

In [ ]:
import numpy as np
from collections import Counter

final_answers = []

for a, b in zip(total_answers, total_results):
    a = np.array(a)
    b = np.array(b)
    print("a:", a)
    a[a < 0] = b[a < 0]

    pred = Counter(a.tolist()).most_common(2)
    print("Pred:", pred)
    try:
        ans = pred[0][0] if not pred[0][0] < 0 else pred[1][0]
    except:
        if len(a) == 1:
            ans = a[0]
        else:
            ans = 0

    final_answers.append(ans)
    print(ans)

In [ ]:
df['answer'] = final_answers

In [ ]:
df

In [ ]:
df[['id','answer']].to_csv("submission.csv", header=True, index=False)

In [ ]:
df[['id','answer']].head()